<div class="alert alert-block alert-warning">
<b>Caution:</b> This notebook uses web scraping to obtain data from the US State Department website. The legality of web scraping varies across the world. In general, web scraping may be against the terms of use of some websites.
</div>

<div class="alert alert-block alert-danger">
<b>Disclaimer:</b> This notebook demonstrates an example of how to use python for data visualization and is solely for educational purposes. It should not be relied upon for making travel decisions. Please visit the US State Department website for upto date travel related information and advisories.
</div>

### Import Libraries

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
import json
import datetime
from datetime import datetime as dt
import pandas as pd

### Define Variables

In [ ]:
# Create lists for each variable
lstCountries = []
lstUrls = []
lstLevel = []
lstDates = []
FILE_LOCATION = ''
URL = ''
IMG_PATH = ''

### Country flag Image location and CSV file used to get country coordinates

<div class="alert alert-block alert-info">
<b>Note:</b> Uncomment the lines of code in the following cell after instering the path to csv files and images and before running the notebook.
</div>

In [ ]:
# FILE_LOCATION = r'[Insert path to your csv file containing country coordinates]'
# IMG_PATH = r'[Insert path to folder holding flag images]'

### Get the __[US State Department](https://travel.state.gov)__ Travel Advisories Data

In [ ]:
URL = 'https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find(class_='table-data')
table_rows = results.tbody.find_all("tr")[1:]
myList = []

In [ ]:
def myconverter(o):
    if isinstance(o,datetime.datetime):
        return o.__str__()

In [ ]:
for tr in table_rows:
        tds = tr.find_all('td')
        for td in tds:
            link = td.find('a')
            try:
                if 'href' in link.attrs:
                    l = link.get('href')
            except:
                pass
        country = tds[0].text.replace('Travel Advisory','').strip()
        base_url = 'https://travel.state.gov'
        url = base_url + l.strip()
        level = tds[1].text
        date = tr.get('data-date')
        date_obj = datetime.datetime.strptime(date,'%m/%d/%y')
        date_converted = myconverter(date_obj)
        date_short = date_converted[:10]
        obj = {"country": country, "hyperlink": url, "level": level, "date": date_short}
        myList.append(obj)

<div class="alert alert-block alert-info">
<b>Note</b> You can also use geolocation services like positionstack.com to get country data including the coordinates.You will need an API key to use their services. However for this small example, using an API service will be an overkill. 
</div>

### Get country coordinates from the CSV File

In [ ]:
# Read country coordinates from the CSV file and store them in an array
lstCountryCoordsArr = []
with open(FILE_LOCATION, newline='') as f:
    reader = csv.reader(f)
    lstCountryCoordsArr = list(reader)


In [ ]:
def getThreats(link):
    countryPage = str(link).strip()
    response = requests.get(countryPage)
    content = BeautifulSoup(response.content, 'html.parser')
    container = content.find(id = 'container tooltipalert')
    anchors = container.find_all(class_="showThreat")
    threats = []
    
    if len(anchors) > 0:
        html = '<div><ul>'
        for a in anchors:
            #print(a.attrs['data-tooltip'])
            data = a.attrs['data-tooltip'].split(':')[0]
            threats.append(data)
            threats_sorted = sorted(threats)
        for threat in threats_sorted:
            html += '<li>' + str(threat) + '</li>'
        html += '</ul></div>'
    else:
        html = '<p style="text-align:center;">None 😎</p>'
    return html

In [ ]:
mapObj = ''

In [ ]:
# import the library
import folium
import base64
from pathlib import Path
imagePath = str(Path(IMG_PATH))
mapObj = folium.Map(location=[20,0], tiles="OpenStreetMap",zoom_start=3)

for i in range(0,len(myList)):
    country =''
    url = ''
    msgNumber =''
    msgLevel = ''
    tooltip = ''
    advisoryDate = ''
    lat = ''
    lng = ''
    file = ''
    html = ''
    icon = ''
    for j in lstCountryCoordsArr:
        if myList[i]['country'].lower() == j[3].lower():
            country = myList[i]['country']
            url = myList[i]['hyperlink']
            msgNumber = myList[i]['level'][:7]
            #print(msgNumber)
            if msgNumber == "Level 2":
                color="green"
                icon = 'fa-exclamation-triangle'
            elif msgNumber == "Level 3":
                color="blue"
                icon = 'fa-question-circle'
            elif msgNumber == "Level 4":
                color="red"
                icon = 'fa-ban'
            else:
                color="darkblue"
            file = imagePath + '\\' + country + '.png'
            advisoryDate =  myList[i]['date']
            msgLevel = '<span style="color:darkblue;">' + myList[i]['level'][9:] +'</span>'
            try:
                encoded = base64.b64encode(open(file, 'rb').read()).decode()
                imgUrl = '<img src="data:file;base64,{}" alt="flag Image">'.format
                html = '<div style="text-align:center;">' + imgUrl(encoded) + '&nbsp;' + '<span style="font-size: 16px; color: darkblue;">'+ '&nbsp;' + country + '</span></div>' + '<p style="text-align:center;"><a style="text-decoration:none;" href=' + '"' + url + '" target=_blank>Travel Advisory</a><br />' + advisoryDate + '</p>'
                tooltip = '<p style="text-align:center;"><span style="font-size:14px;">' + imgUrl(encoded) + '&nbsp;' + country + '</span>' + '<br />' + '<span style="font-size:16px; color:cadetblue;">'+ msgNumber + '</span>' + '<br />' + msgLevel + '</p>'
            except FileNotFoundError:
                    #print('File Not Found')
                    html = '<div style="text-align:center;">' + '<span style="font-size: 16px; color: darkblue;">'+ '&nbsp;' + country + '</span></div>' + '<p style="text-align:center;"><a style="text-decoration:none;" href=' + '"' + url + '" target=_blank>Travel Advisory</a><br />' + advisoryDate + '</p>'
                    tooltip = '<p style="text-align:center;"><span style="font-size:14px;">' + country + '</span>' + '<br />' + '<span style="font-size:16px; color:cadetblue;">'+ msgNumber + '</span>' + '<br />' + msgLevel + '</p>'
            lat = j[1]
            lng = j[2]
            iframe = html + '<p style="background-color:red;border-radius: 2px;text-align:center;font-size:14px; color:white;">Current Threats 💀</p>' + getThreats(url)
            popup = folium.Popup(iframe,min_width=150,max_width=300)
            folium.Marker(location=[lat, lng],tooltip = tooltip,popup= popup,icon=folium.Icon(color=color,icon=icon,prefix='fa')).add_to(mapObj)
            break

        

In [ ]:
#Show Map
mapObj